# Example on how to use Python to read the output files from ASPECT.

We start by importing the packages needed and the functions from the file read_files.py:

In [ ]:
%matplotlib inline  
## This allows figures to be plotted "inline". 
## In a normal python script, you use plt.show() to see the figures, 
## or plt.savefig("name.pdf") to save the figures 
## (you can also use this command in the notebooks)

import numpy as np  # for mathematical functions and arrays
import matplotlib.pyplot as plt  # for plot
import pandas as pd # for data analysis
from scipy.interpolate import griddata
## We use short names for the libraries we use a lot.

import io 
import os
from subprocess import run

import sys 
sys.path.insert(0, '../../scripts/')
import aspect_data

# Running aspect from the notebook 

You may want to modify the path for the aspect build folder. 

A run will provide the output files that we are then reading later. 

In [ ]:
input_file = "convection-box.prm"     
aspect_bin = "../../../../build/aspect"          # Path to aspect executable
output_dir = "./output/"                      # Output directory

def run_aspect():
    run(['rm','-rf',output_dir])
    run([aspect_bin,input_file])
    
run_aspect()

# Parameter file

Sometimes, it's useful to be able to access the values for the parameters inside the parameter file, especially if you are exploring a large parameter space. 

You can read the parameter file with the function read_files.read_parameter_file(filename)

In [ ]:
param_file = output_dir+"parameters.prm"
param = aspect_data.read_parameter_file(param_file)

The param variable is then a dictionary that include all the parameters that have been defined in the original parameter file. One can then access the values using the same names than the subsections in the .prm file. 

In [ ]:
print(param["Material model"])
print(param["Material model: Simple model"])

## File statistics

The file statistics regroup all the statistics for everytime steps. It can be a very long file, so pandas (a package for data analysis) is the best tool to open and read it. 
The number of columns depends on your run, and is indicated as comments at the beginning of the file. The function read_statistics() will first read the comments, extracts the names of the columns and then read the file with pd.read_csv(). 

Here, we use an example of a run with a 1D column. All the output files are in the folder ./example. 

In [ ]:
# we provide the name for the file statistics:
statistics = output_dir+ "statistics"

# we read the file with our function:
data =  aspect_data.read_statistics(statistics)

The output of the function is a pandas dataframe, where all data are in a big table where the columns have human-readable names. We can have a look at the table wit data.head()

In [ ]:
data.head()

We can then look at the evolution of some of the statistics with time: 

In [ ]:
# using the plot function embedded in the panda frame:
data.plot("Time (seconds)", "RMS velocity (m/s)")

In [ ]:
# or using the pyplot functions (allows more flexibility)

fig, ax = plt.subplots()

ax.plot(data["Time (seconds)"], data["RMS velocity (m/s)"], label="RMS velocity")
ax.plot(data["Time (seconds)"], data["Max. velocity (m/s)"], label="Max velocity")
ax.legend()
ax.set_xlabel("Time (s)")

## Visualization files

The statistics file includes when visualization files have been created. If they have been written as gnuplot files, then we can read them and plot them. 

In the example case, the grid is a column of 1 cell width. So the output can be plotted as 1D variable. 

In [ ]:
# we remove the values from data with NaN in the visualization file name column
data = data.dropna(subset=["Visualization file name"])

# we store the values for the file names and the associated time. 
visu_files = data["Visualization file name"].values
visu_times = data["Time (seconds)"].values

df = aspect_data.read_gnuplot_visu(visu_files[0]+".0000.gnuplot")
print(df.head())

### Unstructured grid 

The points are not always on a structured grid, so tricontourf may be the best way to plot the temperature field for example

In [ ]:
fig, ax = plt.subplots(1, 5, sharey=True, sharex=True, figsize=[15,3])

i=0
for time, file in zip(visu_times[:-2:2], visu_files[:-2:2]):
    #print(time, file)
    file = file+".0000.gnuplot"
    df = aspect_data.read_gnuplot_visu(file)
    #df = df[df["x"]==0.5]
    x, y, t = df["x"].values, df["y"].values, df["T"].values
    ax[i].tricontourf(x, y, t, 15, vmin=0, vmax=1)
    i += 1

### Structured grid 

However, for plotting the velocities, it may be easier to interpolate on a structured grid, using griddata from scipy.

This also allows to use streamplots, for example. 

In that case, we choose where the points will be interpolated, which may be better in the case of very asymmetric x/y axis (for example if one axis in log scale and not the other)

In [ ]:
fig, ax = plt.subplots()

# define grid.
xi = np.linspace(0, 1, 100)
yi = np.linspace(0, 1, 100)
xi, yi = np.meshgrid(xi, yi)

# interpolate the values on the grid for the temperature.
Ti = griddata((x, y), t, (xi, yi))
# contour the gridded data, plotting dots at the nonuniform data points.
#CS = plt.contour(xi, yi, Ti, 15, linewidths=0.5, colors='k')
CS = ax.contourf(xi, yi, Ti, 15,
                  vmax=1., vmin=0.)
fig.colorbar(CS, ax=ax, ticks=[0, 0.5, 1])

# grid the velocities
vx, vy = df["velocity_x"].values, df["velocity_y"].values
Vxi = griddata( (x, y), vx, (xi, yi))
Vyi = griddata( (x, y), vy, (xi, yi))
ax.streamplot(xi, yi, Vxi, Vyi, density=[0.5,0.5], color='k')

ax.axis('equal')  # force the x and y axes to have the same scale
ax.set_xlim([0,1])
ax.set_ylim([0,1])
ax.axis('off') # remove the axes and ticks